# **Prontuario per la PI di ML - Giovanni Luca Palombella**

## Machine Learning Classico - `scikit-learn`

https://colab.research.google.com/drive/1964BhFJIc5vf73IL4uUU7LVVM-CWHa6j?authuser=1#scrollTo=pQnjQVP9K2Vs

consulta questo further information

In [ ]:
#@title `LocalOutlierFactor`
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
import seaborn as sns

n_neighbors=20 # iperparametro del LocalOutlierFactor (funziona come un K-NN)

data=df.copy() # copia del mio dataset
clf=LocalOutlierFactor(n_neighbors=n_neighbors) # unico iperparametro da settare
out_in=clf.fit_predict(data[features[:-1]].to_numpy()) # non gli passo le label
count_outliers=np.sum(out_in==-1)
out_index=np.where(out_in==-1)[0]
clean = data[~data.index.isin(out_index)] # rimuovi gli outlier certi [a rigore potrei anche impostare una soglia di tolleranza]
sns.pairplot(clean) # plot
plt.show()

print("Nel dataset sono stati trovati",count_outliers,"outliers.")

In [ ]:
#@title Feature importance con Random Forest e XGBoost - supervised [devo avere le label]
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance

# RandomForest
forest = RandomForestRegressor(n_estimators=250, random_state=0)
forest.fit(X_train, Y_train)
importances_rf = forest.feature_importances_
std_rf = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices_rf = np.argsort(importances_rf)[::-1]

# XGBoost
model_xgb = XGBRegressor()
model_xgb.fit(X_train, Y_train)
importances_xgb = model_xgb.feature_importances_
indices_xgb = np.argsort(importances_xgb)[::-1]

# Plot
plt.figure(figsize=(10, 6))
plt.title("Importanza delle feature - Random Forest vs XGBoost")

bar_width = 0.35
index = np.arange(X.shape[1])

bars_rf = plt.bar(index, importances_rf[indices_rf], bar_width, color='r', yerr=std_rf[indices_rf], alpha=1, label='Random Forest')
bars_xgb = plt.bar(index + bar_width + 0.1, importances_xgb[indices_rf], bar_width, color='b', alpha=1, label='XGBoost')

plt.xlabel('Feature')
plt.ylabel('Importanza')
plt.xticks(index + bar_width / 2, indices_rf)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='upper right')

for bars in [bars_rf, bars_xgb]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, '{:.3f}'.format(height), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
#@title KNN
importances_cut=importances_rf[:2]
def weighted_distance(a,b): # metrica personalizata pesata con la feature importance
  dist = 0
  for i in range(len(a)):
    dist += importances_cut[i]*(a[i]-b[i])**2
  return np.sqrt(dist)

k_values = [ i + 1 for i in range(15)]
best_k = None
scores_train = []
scores_vali = []
ms = 0

for k in k_values: # qui c'è il cuore dell'allenamento
    model = KNeighborsRegressor(n_neighbors=k , metric=weighted_distance)
    model.fit(X_train_norm_cut, Y_train)
    scores_train.append(model.score(X_train_norm_cut,Y_train))
    scores_vali.append(model.score(X_vali_norm_cut, Y_vali))
    if model.score(X_vali_norm_cut, Y_vali) > ms:
      ms = model.score(X_vali_norm_cut, Y_vali)
      best_k = k

print("Il miglior valore di k è:", best_k)

start_time = time.time()

KNN = KNeighborsRegressor(n_neighbors=best_k,metric=weighted_distance)
KNN.fit(X_train_norm_cut, Y_train)

end_time = time.time()
time_KNN = end_time - start_time

plt.title('Scelta del parametro k')
plt.scatter(k_values , scores_train , label='train set',color="red")
plt.scatter(k_values , scores_vali, label='validation set',color="blue")
plt.xlabel('k')
plt.ylabel('$R^2$ score')
plt.grid(color='gray', linewidth=0.5)
plt.xticks(k_values)
plt.legend()
plt.savefig("KNN.png", dpi=500)
plt.show()

predicted = KNN.predict(X_test_norm_cut)

sns.set(style="whitegrid")
sns.scatterplot(x=predicted, y=Y_test, color='skyblue' , s=10)
plt.title("Confronto tra Valori Predetti e Valori Reali")
plt.plot(np.linspace(0,3.7,100),np.linspace(0,3.7,100),linestyle='--')
plt.xlabel("Valori Predetti")
plt.ylabel("Valori Reali")
plt.show()

plt.figure(figsize=(8,7))
error = Y_test - predicted
plt.figure(figsize=(10, 7))
sns.histplot(error, kde=False, color='skyblue', bins=optbins(error))
mean_error = np.mean(error)
std_error = np.std(error)
plt.axvline(x=mean_error, color='red', linestyle='--', label='Media')
plt.axvline(x=mean_error + std_error, color='orange', linestyle='--', label='Media + Std')
plt.axvline(x=mean_error - std_error, color='orange', linestyle='--', label='Media - Std')
plt.title('Distribuzione dell\'errore di previsione')
plt.xlabel('Errore')
plt.ylabel('Frequenza')
plt.legend()
plt.show()

print("Il punteggio sul test set è di", KNN.score(X_test_norm_cut,Y_test))
print("Il tempo di esecuzione è di", time_KNN)
print("MSE = ", mse(Y_test,predicted))

error_KNN = error
best_k_KNN = best_k
scores_vali_KNN = scores_vali
scores_train_KNN= scores_train
predicted_KNN = predicted

In [ ]:
#@title Clustering using Gaussian Mixture

from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
gmm.fit(data_norm[features].values)

y_pred = gmm.predict(data_norm[features].values)

#procedura: grid nello spazio PCA e poi trasformazione PCA inversa per
#tornare nello spazio originale al quale applicare la predizione della gmm
h = .02

xx, yy = np.meshgrid(np.arange(-4., 4., h),
                     np.arange(-4., 4., h))

zz = np.c_[xx.ravel(), yy.ravel()]

zz_orig = pca.inverse_transform(zz)
prob = gmm.predict_proba(zz_orig)

prob = prob.reshape((xx.shape[0],xx.shape[1],3))

plt.contour(xx, yy, prob[:,:,0], levels=[0.68], colors='blue')
plt.contour(xx, yy, prob[:,:,1], levels=[0.68], colors='brown')
plt.contour(xx, yy, prob[:,:,2], levels=[0.68], colors='green')

plt.scatter(pca_result[:,0], pca_result[:,1], c=y_pred, cmap='tab10')
plt.xlabel('PCA_0')
plt.ylabel('PCA_1')
plt.colorbar()

## Neural Networks - `pyTorch`

In [7]:
#@title Import di pyTorch
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

from torch import nn
import torch.nn.functional as F

2.3.0+cu121
0.18.0+cu121


In [ ]:
#@title Check GPU e set device
# controlliamo se la GPU è disponibile e nel caso quale tipo di GPU
if torch.cuda.is_available():
  print('Numero di GPU disponibili: ',torch.cuda.device_count())
  for i in range(0,torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

# se la GPU è disponibile setto device='cuda', altrimenti 'cpu
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

In [ ]:
#@title From numPy to Torch

X_train_pt = torch.from_numpy(X_train).float()
Y_train_pt = torch.from_numpy(Y_train).int() # le label sono dei tipi interi
X_test_pt = torch.from_numpy(X_test).float()
Y_test_pt = torch.from_numpy(Y_test).int()

# reshape the tensori da (n,784) a (n,1,28,28) come richiesto da conv layer in pytorch - sono una caterva ['lunga lista'] di matrici 28x28x1 IMMAGINA UNA LISTA DI MATRICI

X_train_pt = X_train_pt.reshape(len(X_train_pt), 1, 28,28) #
X_test_pt = X_test_pt.reshape(X_test_pt.shape[0], 1, 28, 28) # questa è la soluzione di Giagu

print(X_train_pt.shape)
print(Y_train_pt.shape)
print(X_test_pt.shape)
print(Y_test_pt.shape)

In [ ]:
#@title Saver per checkpoint
class SaveBestModel:
    def __init__(self, best_valid_loss=float('inf')):
        self.best_valid_loss = best_valid_loss

    def __call__(
      self, current_valid_loss,
      epoch, model, optimizer, criterion):
      #Se non vogliamo usare il validation salviamo il modello ogni 5 epoche (eventualmente si può cambiare il numero)
      #Lo tengo perchè la validation loss diminuisce ad ogni epoca
      if epoch%5:
        return 0

      if current_valid_loss < self.best_valid_loss:
          self.best_valid_loss = current_valid_loss

          print(f"\n[+] Best validation loss: {self.best_valid_loss}")
          print(f"[->] Saving best model for epoch: {epoch+1}")

          torch.save({'model' : model,
              'epoch': epoch+1,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': criterion,
              }, 'best_model.pt')

saver = SaveBestModel() # così va istanziato e poi lo chiamo ogni volta all'interno del training loop

In [ ]:
#@title Training loop
import time

num_epochs = 20
l1_lambda = 0.0001 # iperparametro del regolarizzatore LASSO - va bene circa un ordine in meno del learning rate
hist_loss = []
hist_vali_loss = []

for epoch in range(num_epochs):
  t0 = time.time()

  #Training
  model.train()
  train_loss = 0
  counter = 0
  for x,_ in dataloaders['train']:
    #Spostiamo il tensore sul device
    x = x.to(device)

    #Creiamo la ricostruzione
    output = model(x)

    #Aggiungiamo regolarizzione L1 Penalty
    l1_penalty = 0
    for param in model.parameters():
      l1_penalty += torch.sum(torch.abs(param))

    #Calcoliamo e aggiorniamo la loss per questa epoca
    temp_loss = Loss_fn(output, x)+l1_penalty*l1_lambda # qui calcola la loss
    train_loss += temp_loss.item()
    counter+=1

    #Backpropagation [FA LE DERIVATE]
    opt.zero_grad()
    temp_loss.backward()
    opt.step()

  #Salviamo la loss di questa epoca
  train_loss /= counter
  hist_loss.append(train_loss)

  '''
  #Validation
  vali_loss = 0
  counter = 0
  model.eval()
  with torch.no_grad(): # importantissimo!
    for x, _ in dataloaders['test']:
      #Spostiamo il tensore sul dispositivo in utilizzo
      x = x.to(device)

      #Ricostruzione dell'input
      output = model(x)

      #Calcolo della loss
      temp_loss = Loss_fn(output, x)
      vali_loss += temp_loss.item()
      counter+=1

  #Aggiornamento della vali loss
  vali_loss/=counter
  hist_vali_loss.append(vali_loss)
  '''

  # check point
  saver(vali_loss, epoch, model_ae, opt, Loss_fn)
  #         ^---------Va cambiato in train_loss in caso niente validation

  #Update dello scheduler - nel caso sia presente
  scheduler.step()

  #Calcolo del tempo impiegato
  elapsed_time = time.time() - t0

  #Logging del risultato
  print(f'[+] epoch : {epoch+1:4d} | Time : {elapsed_time:.4f} | Train Loss : {train_loss:.8f} | Vali Loss : {vali_loss:.8f}')


## Misc

In [5]:
#@title Come leggere database *.csv
import numpy as np
import pandas as pd
nomefile = '/content/sample_data/mnist_test.csv' # inserire qui il percorso del file
df = pd.read_csv(nomefile)

In [ ]:
#@title Funzione per numero ottimale di bins
def optbins(lista):
  range=np.max(lista)-np.min(lista)
  return int(range*(len(lista)**(1./3.))/(3.49*np.std(lista)))

In [ ]:
#@title Split in T-V-T set - *allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes*
from sklearn.model_selection import train_test_split
train_to_rest_ratio=0.6 # training sample 60%
vali_to_test_ratio=0.5  # validation and test sets: 50%-50% of remainining 40%

# selezioniamo random data points dai campioni ordinati e disordinati per creare training, test e validation.
# shuffle=True dato che i dati sembrano ordinati
X_train,X,Y_train,Y=train_test_split(clean[features], clean['target'],train_size=train_to_rest_ratio, shuffle=True, random_state=21)
X_vali,X_test,Y_vali,Y_test=train_test_split(X,Y,train_size=vali_to_test_ratio, shuffle=True, random_state=21)

In [ ]:
#@title Matrice di correlazione
corr_cleaned = clean.corr() # clean è un dataframe di pandas
plt.figure(figsize=(8, 7))
sns.heatmap(corr_cleaned, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title('Matrice di correlazione')
plt.show()

In [ ]:
#@title Confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
confusion_mat = confusion_matrix(y_test.astype("int"), pred)
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues')

In [ ]:
#@title Normalizzazione dei sottocampioni - LA NORMALIZZAZIONE VA FATTA INDIPENDENTEMENTE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_train_norm = pd.DataFrame(X_train_norm, columns=X_train.columns)

X_vali_norm = scaler.transform(X_vali)
X_vali_norm = pd.DataFrame(X_vali_norm, columns=X_vali.columns)

X_test_norm = scaler.transform(X_test)
X_test_norm = pd.DataFrame(X_test_norm, columns=X_test.columns)

plt.scatter(X_train_norm['x_0'],X_train_norm['x_1'], label='X_train normalizzato',color="blue")
plt.scatter(X_train['x_0'],X_train['x_1'], label='X_train', color='red')
plt.axis('equal')
plt.grid(True)
plt.xlabel("$x_0$")
plt.ylabel("$x_1$")
plt.title("Dati delle feature $x_0$ e $x_1$ prima e dopo la normalizzazione")
plt.legend()

In [ ]:
#@title Class balance plot
df['class'].value_counts(normalize=True).to_frame().style.bar(color='green')

In [ ]:
#@title Trasformare label da string a indice numerico intero (0,1,2)
from sklearn import preprocessing

LE = preprocessing.LabelEncoder() #funzione automatica di scikit learn
df['class'] = LE.fit_transform(df['class']) #sostituisce agli elementi della colonna class gli interi che trova in automatico, le etichette le mette lui e non ce ne frega un cazzo
df['class'].value_counts()

COGLIONE!!! PRIMA DI FARE LA PCA DEVI NORMALIZZARE

In [ ]:
#@title PCA
from sklearn.decomposition import PCA

pca2 = PCA(n_components=10)

pca2_result = pca2.fit_transform(X_mnist) # dare in input un vettore numpy

exp_var2 = pca2.explained_variance_ratio_

for i in range(0,len(exp_var2)):
  print('explained variance componente: ',i, ' = ', exp_var2[i])
print('Explained variance totale delle prime 10 componenti della PCA: {}'.format(sum(exp_var2)))

import matplotlib.pyplot as plt
plt.figure(figsize=(7,5))
plt.scatter(pca2_result[:,0], pca2_result[:,1], c=Y_mnist, cmap='tab10') # colorare con le label
cbar = plt.colorbar()
cbar.set_label('Classe')
plt.xlabel('$PCA_0$')
plt.ylabel('$PCA_1$')


# Xtransf_mnist = pca2.inverse_transform(pca2_result[:16]) # trasformazione inversa

Esercitazione 5 sulla riduzione dimensionale [TSNE, PCA, uMAP]

https://colab.research.google.com/github/stefanogiagu/corso_AI_2024/blob/main/notebooks/es5/MetodiAI_Fis2024_Es5_PCA_tSNE_UMAP.ipynb?authuser=1#scrollTo=BYfpAkW4G44e

Oss: uMAP è molto pesante e non preserva le distanze; tuttavia separa molto bene.


Nella prima parte della esercitazione 6 ci sono degli hints interessanti per quanto riguarda il lavorare con le feature prese da tabelle pandas.